# Employee retention

One of the challenges from ["Collection of Data Science Take-home Challenges"](http://datascientistjobinterview.com/).

I worked on this with the [Central London Data Science Project Night](http://www.meetup.com/central_london_data_science/) meetup group.

In [1]:
import numpy as np
import pandas as pd

In [2]:
original_df = pd.read_csv('./employee_retention_data.csv')

## Fix types and observe data

In [3]:
original_df.describe()

,employee_id,company_id,seniority,salary
count,24702.000000,24702.000000,24702.000000,24702.000000
mean,501604.403530,3.426969,14.127803,138183.345478
std,288909.026101,2.700011,8.089520,76058.184573
min,36.000000,1.000000,1.000000,17000.000000
25%,250133.750000,1.000000,7.000000,79000.000000
50%,500793.000000,2.000000,14.000000,123000.000000
75%,753137.250000,5.000000,21.000000,187000.000000
max,999969.000000,12.000000,99.000000,408000.000000


In [4]:
original_df.columns

Index(['employee_id', 'company_id', 'dept', 'seniority', 'salary', 'join_date',
       'quit_date'],
      dtype='object')

In [5]:
original_df.shape

(24702, 7)

In [6]:
df = original_df.copy()

In [7]:
def date_str_to_date(d):
    if isinstance(d, float):
        return None
    else:
        return pd.datetime(*map(int, d.split('-')))

In [8]:
df['company_id'] = original_df['company_id'].apply(str)
df['employee_id'] = original_df['employee_id'].apply(int).apply(str)
df['join_date'] = original_df['join_date'].apply(date_str_to_date)
df['quit_date'] = original_df['quit_date'].apply(date_str_to_date)

In [9]:
df.describe()

,seniority,salary
count,24702.000000,24702.000000
mean,14.127803,138183.345478
std,8.089520,76058.184573
min,1.000000,17000.000000
25%,7.000000,79000.000000
50%,14.000000,123000.000000
75%,21.000000,187000.000000
max,99.000000,408000.000000


In [10]:
df.dtypes

employee_id            object
company_id             object
dept                   object
seniority               int64
salary                float64
join_date      datetime64[ns]
quit_date      datetime64[ns]
dtype: object

In [11]:
type(df['join_date'][0])

pandas.tslib.Timestamp

In [12]:
df.head()

,employee_id,company_id,dept,seniority,salary,join_date,quit_date
0,13021,7,customer_service,28,89000.0,2014-03-24,2015-10-30
1,825355,7,marketing,20,183000.0,2013-04-29,2014-04-04
2,927315,4,marketing,14,101000.0,2014-10-13,NaT
3,662910,7,customer_service,20,115000.0,2012-05-14,2013-06-07
4,256971,2,data_science,23,276000.0,2011-10-17,2014-08-22


In [13]:
df.shape

(24702, 7)

In [14]:
for c in df.columns:
    print('unique {}: {}'.format(c, df[c].unique().shape[0]))

unique employee_id: 24702
unique company_id: 12
unique dept: 6
unique seniority: 31
unique salary: 367
unique join_date: 995
unique quit_date: 665


In [15]:
departments = df['dept'].unique()
departments

array(['customer_service', 'marketing', 'data_science', 'engineer',
       'sales', 'design'], dtype=object)

## Table transformation challenge

Estimate employee headcount, for each company, on each day, from 2011/01/24 to 2015/12/13. 


|     day    | employee_headcount | company_id |
|:----------:|:------------------:|:----------:|
| 2011-01-24 |                  ? |          1 |
| 2011-01-25 |                  ? |          1 |
| 2011-01-26 |                  ? |          1 |
| ... | ... | ... |
| 2015-12-13 |                  ? |         12 |


In [16]:
range_start = pd.datetime(2011, 1, 24)
range_end = pd.datetime(2015, 12, 13)
date_range = pd.date_range(range_start, range_end, name='day')
date_range[:3]

DatetimeIndex(['2011-01-24', '2011-01-25', '2011-01-26'], dtype='datetime64[ns]', name='day', freq='D')

In [17]:
print('number of days: {}'.format(date_range.shape[0]))

number of days: 1785


In [18]:
def day_index(date):
    delta = date - range_start
    return delta.days

day_index(pd.datetime(2011, 1, 31))

7

In [19]:
# how can I go through all pairs of successive items in a list (in order)

l = range(5)
print(list(l))

for first, second in zip(l[:-1], l[1:]):
    print((first, second))

[0, 1, 2, 3, 4]
(0, 1)
(1, 2)
(2, 3)
(3, 4)


In [20]:
company_dfs = []

for company_id, company_data in df.groupby('company_id'):
    
    # get all events (join or quit) with the date
    join_dates = company_data['join_date']
    quit_dates = company_data['quit_date'].dropna()
    print('company {}: {} joined, {} left.'.format(company_id, join_dates.shape[0], quit_dates.shape[0]))
    all_dates = sorted([(d, 'J') for d in join_dates] + [(d, 'Q') for d in quit_dates])
    
    # for each date where something happened, compute the headcount
    headcounts = [(range_start, 0)]
    prev_date = range_start
    prev_headcount = 0
    for d, event in all_dates:
        delta = 1 if event == 'J' else -1
        new_headcount = prev_headcount + delta
        prev_headcount = new_headcount
        if prev_date == d:
            # do not duplicate the date, instead merge the events
            headcounts[-1] = (d, new_headcount)
        else:
            headcounts.append((d, new_headcount))
            prev_date = d
    headcounts.append((range_end, headcounts[-1][1]))
    
    by_day = pd.DataFrame({'day': date_range, 'headcount': -1})
    # go through pairs of successive headcounts
    for (d, count), (next_d, _) in zip(headcounts[:-1], headcounts[1:]):
        by_day.loc[day_index(d):day_index(next_d), 'headcount'] = count
    
    by_day['company_id'] = company_id
    
    company_dfs.append(by_day)


company 1: 8486 joined, 4621 left.
company 10: 865 joined, 480 left.
company 11: 16 joined, 12 left.
company 12: 24 joined, 12 left.
company 2: 4222 joined, 2206 left.
company 3: 2749 joined, 1531 left.
company 4: 2062 joined, 1153 left.
company 5: 1755 joined, 983 left.
company 6: 1291 joined, 712 left.
company 7: 1224 joined, 692 left.
company 8: 1047 joined, 579 left.
company 9: 961 joined, 529 left.


In [21]:
employee_headcount_df = pd.concat(company_dfs)
employee_headcount_df.shape

(21420, 3)

In [22]:
employee_headcount_df.head()

,day,headcount,company_id
0,2011-01-24,25,1
1,2011-01-25,27,1
2,2011-01-26,29,1
3,2011-01-27,29,1
4,2011-01-28,29,1


In [23]:
employee_headcount_df.tail()

,day,headcount,company_id
1780,2015-12-09,432,9
1781,2015-12-10,432,9
1782,2015-12-11,432,9
1783,2015-12-12,432,9
1784,2015-12-13,432,9


## Employee churn

What are the main factors that drive employee churn?

In [24]:
df.columns

Index(['employee_id', 'company_id', 'dept', 'seniority', 'salary', 'join_date',
       'quit_date'],
      dtype='object')

In [25]:
df['employment_duration'] = df['quit_date'] - df['join_date']

In [26]:
df['employment_duration'].describe()

count                       13510
mean     613 days 11:41:01.643227
std      328 days 14:56:33.800149
min             102 days 00:00:00
25%             361 days 00:00:00
50%             417 days 00:00:00
75%             781 days 00:00:00
max            1726 days 00:00:00
Name: employment_duration, dtype: object

**idea:**
linear regression using `dept`, `seniority`, `salary`, `join_date` to predict `employment_duration`. Check the weights.

**TODO**